In [71]:
from datasets import load_dataset
from langchain.document_loaders import DataFrameLoader
import csv
import json

In [35]:
dataset = load_dataset("squad_v2")

Found cached dataset squad_v2 (/home/melek/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [4]:
for split, dataset in dataset.items():
    dataset.to_csv(f"squad-{split}.csv")

Creating CSV from Arrow format:   0%|          | 0/131 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

In [125]:
def get_rows(df):
    docs = DataFrameLoader(df, page_content_column="context").load()
    docs = [{'page_content':doc.page_content, 'metadata':doc.metadata} for doc in docs]
    
    for doc in docs:
        doc['metadata']['answer'] = next(iter(doc['metadata']['answers']['text']), None) # select first anwser or None
        doc['metadata'].pop('answers') # delete answers
    
    return [{'page_content':doc['page_content'], 'metadata':json.dumps(doc['metadata'])} for doc in docs]

In [128]:
def write_csv(file, rows):
    with open(file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['page_content','metadata'])
        writer.writeheader()
        writer.writerows(rows)

In [129]:
write_csv('squad-train-docs.csv', get_rows(dataset['train'].to_pandas()))

In [130]:
write_csv('squad-validation-docs.csv', get_rows(dataset['validation'].to_pandas()))